## 2021: Week 37 - Re-looking at Phone Contract Revenue

The latest version of Tableau Prep was released this week! This included a great new feature that allows you to generate new rows that aren't already available within your data set. If you want to know more then check out Carl's blog post where he has this covered!

As there's a new feature that makes our lives that little bit easier, I thought it would be a great opportunity to revisit one of the older challenges to see how things have changed. Therefore this week we are nearly going all the way back to where Preppin' Data began... Challenge 2019 Week 3! 

If you haven't completed this challenge then you can find the original post here, but for the challenge we are going to calculate the recurring revenue based on a mobile phone contract length. This can be completed in the new version of Prep (2021.3 onwards) or an older version if you haven't had the opportunity to download it yet (better yet, why not try both!). 

### Scenario
You work for a mobile / cell phone company. You boss asks you to pull together the revenue report of your current batch of contracts (sadly there are only four contracts!). They need to know how much revenue is generated each month from these contracts whilst they are 'live' (ie from their start date until 'x' months in the future when the contract runs out).

### Requirement
- Input the Data
- Calculate the End Date for each contract
- Create a Row for each month a person will hold the contract
- Calculate the monthly cumulative cost of each person's contract
- Output the Data

### Output
![img](https://1.bp.blogspot.com/-QLr15tWCcxQ/YUBuX_t2CRI/AAAAAAAAHZM/loe_8PMYy8kcofoEYXeC6f5EquPDKJmqACLcBGAsYHQ/s320/Output.png)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Input the Data

In [3]:
data = pd.read_excel("./data/2021 Week 37 Input.xlsx", sheet_name=[0, 1])
contract = data[0].copy()

In [4]:
contract

,Name,Monthly Cost,Contract Length (months),Start Date
0,Carl,20,24,2018-12-13
1,Jonathan,15,6,2019-02-22
2,Andy,45,12,2018-10-17
3,Sophie,30,12,2018-11-19


In [5]:
contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Name                      4 non-null      object        
 1   Monthly Cost              4 non-null      int64         
 2   Contract Length (months)  4 non-null      int64         
 3   Start Date                4 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 256.0+ bytes


### Calculate the End Date for each contract

In [9]:
dt = pd.Timestamp(contract["Start Date"][0])
dt + pd.DateOffset(months=24)

Timestamp('2020-12-13 00:00:00')

In [17]:
contract["End Date"] = contract.apply(lambda x: x["Start Date"] + pd.DateOffset(months=x["Contract Length (months)"]), axis=1)
contract["End Date"]

0   2020-12-13
1   2019-08-22
2   2019-10-17
3   2019-11-19
Name: End Date, dtype: datetime64[ns]

### Create a Row for each month a person will hold the contract